In [1]:
import pandas as pd
import numpy as np

In [2]:
df111 = pd.read_hdf('../input_hdf/train_date_162cols.hdf5','table')

In [3]:
usecol = df111.columns.drop(['Id'])

In [4]:
numeric = "../input_orig/train_numeric.csv"
date = "../input_orig/train_date.csv"
categorical = "../input_orig/train_categorical.csv"

numeric_te = "../input_orig/test_numeric.csv"
date_te = "../input_orig/test_date.csv"
categorical_te = "../input_orig/test_categorical.csv"

In [5]:
def orgainize(features):
    line_features = {}
    station_features = {}
    lines = set([f.split('_')[0] for f in features])
    stations = set([f.split('_')[1] for f in features])
    
    for l in lines:
        line_features[l] = [f for f in features if l+'_' in f]
        
    for s in stations:
        station_features[s] = [f for f in features if s+'_' in f]
            
    return line_features, station_features

In [6]:
features_num = pd.read_csv(numeric, nrows=1).drop(['Response', 'Id'], axis=1).columns.values
line_features_num, station_features_num = orgainize(features_num)
print("Features in Station 32: {}".format( station_features_num['S32'] ))

Features in Station 32: ['L3_S32_F3850']


In [7]:
features_date = pd.read_csv(date, nrows=1).drop(['Id'], axis=1).columns.values
line_features_date, station_features_date = orgainize(features_date)
print("Features in Station 32: {}".format( station_features_date['S32'] ))

Features in Station 32: ['L3_S32_D3852']


In [8]:
features_cat = pd.read_csv(categorical, nrows=1).drop(['Id'], axis=1).columns.values
line_features_cat, station_features_cat = orgainize(features_cat)
print("Features in Station 32: {}".format( station_features_cat['S32'] ))

Features in Station 32: ['L3_S32_F3851', 'L3_S32_F3853', 'L3_S32_F3854']


In [9]:
CHUNKSIZE = 200000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

tr_hash_full_date = pd.DataFrame()
for df_tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32, usecols=usecol):
    tmp1 = pd.DataFrame()
    df_tr.fillna(-1, inplace=True)
    cols = df_tr.columns
    for col in cols:
        df_tr[col] = (1 * (df_tr[col] != -1)).astype(np.int8)
    tmp1['hash_full'] = df_tr.apply(lambda x: hash(tuple(x)), axis = 1)
    tr_hash_full_date = tr_hash_full_date.append(tmp1)

    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break

In [10]:
te_hash_full_date = pd.DataFrame()
for df_te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32, usecols=usecol):
    tmp2 = pd.DataFrame()
    df_te.fillna(-1, inplace=True)
    cols = df_te.columns
    for col in cols:
        df_te[col] = (1 * (df_te[col] != -1)).astype(np.int8)
    tmp2['hash_full'] = df_te.apply(lambda x: hash(tuple(x)), axis = 1)
    te_hash_full_date = te_hash_full_date.append(tmp2)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [11]:
tr_hash_full_date.shape, te_hash_full_date.shape

((1183747, 1), (1183748, 1))

In [12]:
tr_hash_full_date.hash_full.unique().shape

(17017,)

In [35]:
tr_hash_full_date.head()

,hash_full_date
0,0
1,1
2,2
3,3
4,4


In [33]:
tr_hash_full_date.columns = ['hash_full_date']
te_hash_full_date.columns = ['hash_full_date']

In [14]:
hash_full_date = pd.concat([tr_hash_full_date,te_hash_full_date],axis=0,ignore_index=True)

In [22]:
hash_full_date['hash_full'] = pd.factorize(hash_full_date['hash_full'])[0]

In [24]:
hash_full_date.hash_full.max()

23173

In [25]:
tr_hash_full_date = hash_full_date.ix[0:tr_hash_full_date.shape[0]-1]
te_hash_full_date = hash_full_date.ix[tr_hash_full_date.shape[0]:]

In [26]:
te_hash_full_date.reset_index(drop=1,inplace=1)

In [27]:
tr_hash_full_date = tr_hash_full_date.astype(np.int32)
te_hash_full_date = te_hash_full_date.astype(np.int32)

In [36]:
tr_hash_full_date.to_hdf('feats/tr_hash_full_date13.hdf5','table')
te_hash_full_date.to_hdf('feats/te_hash_full_date13.hdf5','table')

In [37]:
tr_num_nan = pd.read_hdf('../input_hdf/train_numeric_928cols_NANs.hdf5','table')
te_num_nan = pd.read_hdf('../input_hdf/test_numeric_928cols_NANs.hdf5','table')

In [40]:
tr_num_nan.drop(['Id'],axis=1,inplace=1)
te_num_nan.drop(['Id'],axis=1,inplace=1)

In [41]:
tr_hash_full_num = tr_num_nan.apply(lambda x: hash(tuple(x)), axis = 1)

In [42]:
te_hash_full_num = te_num_nan.apply(lambda x: hash(tuple(x)), axis = 1)

In [45]:
tr_hash_full_num = pd.DataFrame(tr_hash_full_num, columns=['hash_full_num'])
te_hash_full_num = pd.DataFrame(te_hash_full_num, columns=['hash_full_num'])

In [51]:
hash_full_num = pd.concat([tr_hash_full_num,te_hash_full_num],axis=0,ignore_index=True)

In [54]:
hash_full_num.hash_full_num.unique().shape

(22717,)

In [55]:
hash_full_num['hash_full_num'] = pd.factorize(hash_full_num['hash_full_num'])[0]

In [58]:
tr_hash_full_num = hash_full_num.ix[0:tr_hash_full_num.shape[0]-1]
te_hash_full_num = hash_full_num.ix[tr_hash_full_num.shape[0]:]

In [63]:
te_hash_full_num.reset_index(drop=1,inplace=1)

In [66]:
tr_hash_full_num = tr_hash_full_num.astype(np.int32)
te_hash_full_num = te_hash_full_num.astype(np.int32)

In [67]:
tr_hash_full_num.to_hdf('feats/tr_hash_full_num13.hdf5','table')
te_hash_full_num.to_hdf('feats/te_hash_full_num13.hdf5','table')

In [68]:
tr_cat_205 = pd.read_hdf('../input_hdf/train_categorical_205cols.hdf5','table')

In [70]:
usecol = tr_cat_205.columns.drop(['Id'])

In [87]:
import warnings
warnings.filterwarnings('ignore')

CHUNKSIZE = 50000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

tr_hash_full_cat = pd.DataFrame()
for df_tr in pd.read_csv(categorical, chunksize=CHUNKSIZE, usecols=usecol):
    tmp1 = pd.DataFrame()
    df_tr.fillna(-1, inplace=True)
    cols = df_tr.columns
    for col in cols:
        df_tr[col] = (1 * (df_tr[col] != -1)).astype(np.int8)
    tmp1['hash_full_cat'] = df_tr.apply(lambda x: hash(tuple(x)), axis = 1)
    tr_hash_full_cat = tr_hash_full_cat.append(tmp1)

    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break

In [88]:
te_hash_full_cat = pd.DataFrame()
for df_te in pd.read_csv(categorical_te, chunksize=CHUNKSIZE, usecols=usecol):
    tmp2 = pd.DataFrame()
    df_te.fillna(-1, inplace=True)
    cols = df_te.columns
    for col in cols:
        df_te[col] = (1 * (df_te[col] != 0)).astype(np.int8)
    tmp2['hash_full_cat'] = df_te.apply(lambda x: hash(tuple(x)), axis = 1)
    te_hash_full_cat = te_hash_full_cat.append(tmp2)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [89]:
hash_full_cat = pd.concat([tr_hash_full_cat,te_hash_full_cat],axis=0,ignore_index=True)

In [90]:
hash_full_cat['hash_full_cat'] = pd.factorize(hash_full_cat['hash_full_cat'])[0]

In [91]:
tr_hash_full_cat = hash_full_cat.ix[0:tr_hash_full_cat.shape[0]-1]
te_hash_full_cat = hash_full_cat.ix[tr_hash_full_cat.shape[0]:]

In [92]:
te_hash_full_cat.reset_index(drop=1,inplace=1)

In [93]:
tr_hash_full_cat = tr_hash_full_cat.astype(np.int32)
te_hash_full_cat = te_hash_full_cat.astype(np.int32)

In [94]:
tr_hash_full_cat.to_hdf('feats/tr_hash_full_cat13.hdf5','table')
te_hash_full_cat.to_hdf('feats/te_hash_full_cat13.hdf5','table')

In [95]:
tr_hash_full_date_num_cat = pd.concat([tr_hash_full_date,tr_hash_full_num,tr_hash_full_cat],axis=1)
te_hash_full_date_num_cat = pd.concat([te_hash_full_date,te_hash_full_num,te_hash_full_cat],axis=1)

In [96]:
tr_hash_full_cat.hash_full_cat.unique()

array([   0,    1,    2, ..., 1757, 1758, 1759])

In [97]:
tr_hash_full_date_num_cat.to_hdf('feats/tr_hash_full_date_num_cat13.hdf5','table')
te_hash_full_date_num_cat.to_hdf('feats/te_hash_full_date_num_cat13.hdf5','table')

In [110]:
tr_hash_full_date_num_cat['hash_full_all'] = tr_hash_full_date_num_cat.apply(lambda x: hash(tuple(x)), axis = 1)
te_hash_full_date_num_cat['hash_full_all'] = te_hash_full_date_num_cat.apply(lambda x: hash(tuple(x)), axis = 1)

In [118]:
tr_hash_full_date_num_cat['hash_full_num_cat'] = tr_hash_full_date_num_cat[['hash_full_num','hash_full_cat']].apply(lambda x: hash(tuple(x)), axis = 1)
te_hash_full_date_num_cat['hash_full_num_cat'] = te_hash_full_date_num_cat[['hash_full_num','hash_full_cat']].apply(lambda x: hash(tuple(x)), axis = 1)

In [119]:
hash_full_all = pd.concat([tr_hash_full_date_num_cat,te_hash_full_date_num_cat],axis=0,ignore_index=True)

In [124]:
hash_full_all['hash_full_all'] = pd.factorize(hash_full_all['hash_full_all'])[0]

In [131]:
hash_full_all['hash_full_num_cat'] = pd.factorize(hash_full_all['hash_full_num_cat'])[0]

In [135]:
tr_hash_full_date_num_cat = hash_full_all.ix[0:tr_hash_full_date_num_cat.shape[0]-1]
te_hash_full_date_num_cat = hash_full_all.ix[tr_hash_full_date_num_cat.shape[0]:]

In [141]:
te_hash_full_date_num_cat.reset_index(drop=1,inplace=1)

In [142]:
tr_hash_full_date_num_cat = tr_hash_full_date_num_cat.astype(np.int32)
te_hash_full_date_num_cat = te_hash_full_date_num_cat.astype(np.int32)

In [144]:
tr_hash_full_date_num_cat.to_hdf('feats/tr_hash_full_date_num_cat13.hdf5','table')
te_hash_full_date_num_cat.to_hdf('feats/te_hash_full_date_num_cat13.hdf5','table')